In [1]:
import pandas as pd
import numpy as np
import os
from ipyfilechooser import FileChooser

pd.set_option('display.max_columns', 500)

## Pick and load configuration file

In [2]:
fc1 = FileChooser('')
fc1.show_only_dirs = False
display(fc1)

FileChooser(path='/home/greg/Nextcloud/Q500', filename='', title='', show_hidden=False, select_desc='Select', …

In [3]:
# Display selected folder
print(fc1.selected_filename)

Q500_configuration.xlsx


## Pick and load Skyline Data

In [4]:
fc2 = FileChooser('')
fc2.show_only_dirs = False
display(fc2)

FileChooser(path='/home/greg/Nextcloud/Q500', filename='', title='', show_hidden=False, select_desc='Select', …

In [45]:
# Display selected folder
print(fc2.selected_path)
#print(fc.selected_filename)
print(fc2.selected)
# load skyline data
data = pd.read_csv(fc2.selected)

/home/greg/Nextcloud/Q500
/home/greg/Nextcloud/Q500/6069 Plate 1 Skyline Q500 Export beta_LC1.csv


In [46]:
# check data to see if LC1/LC2 or Sciex/TQXS

if (data['Molecule Name'].str.contains('Met-His').any()):
    data_type = "LC1"
    if (data['Molecule Name'].str.contains('Met-His_2').any()):
        ms_type = "Sciex"
        lc1_df = pd.read_excel(fc1.selected, "LC1")
        analytes = lc1_df['Analyte Peak Name']
        internal_stds = lc1_df['IS Peak Name']
        acq_method = 'MxP500L-LC1_5712'
    else:
        ms_type = "Waters"
        lc1_df = pd.read_excel(fc1.selected, "LC1_TQXS")
        analytes = lc1_df['Analyte Peak Name']
        internal_stds = lc1_df['IS Peak Name']
        acq_method = 'MxP500L-LC1_8312'
else:
    data_type = "LC2"
    if (data['Molecule Name'].str.contains('AA_3').any()):
        ms_type = "Sciex"
        lc2_df = pd.read_excel(fc1.selected, "LC2")
        analytes = lc2_df['Analyte Peak Name']
        internal_stds = lc2_df['IS Peak Name']
        acq_method = 'MxP500L-LC2_5712'
    else:
        ms_type = "Waters"
        lc2_df = pd.read_excel(fc1.selected, "LC2_TQXS")
        analytes = lc2_df['Analyte Peak Name']
        internal_stds = lc2_df['IS Peak Name']  
        acq_method = 'MxP500L-LC1_8312'
        
print(f"Data type is --> {ms_type} {data_type}")
print(f"Method name is --> {acq_method}")

Data type is --> Sciex LC1
Method name is --> MxP500L-LC1_5712


In [47]:
col_names = ['Analyte Peak Name', 'Sample ID', 'Analyte Retention Time (min)', 'Sample Type', 
             'Analyte Peak Area (counts)', 'Area Ratio','Dilution Factor', 'Analyte Concentration (uM)',
             'Calculated Concentration (uM)', 'Accuracy (%)', 'File Name', 'Acquisition Date', 
             'Analyte Peak Height (cps)', 'Analyte Peak Width (min)', 'IS Peak Name', 'IS Retention Time (min)',
             'IS Peak Area (counts)', 'IS Concentration (uM)', 'IS Peak Height (cps)', 'IS Peak Width (min)', 
             'Height Ratio', 'Acquisition Method', 'Analyte Units', 'IS Units', 'Plate Position','Rack Position', 
             'Analyte Mass Ranges (Da)', 'Analyte Signal To Noise', 'IS Signal To Noise', 'Relative Retention Time', 
             'Sample Name']

blank = '11000002'
double_blank = '10000001'

In [48]:
#create blank dataframe
final_data = pd.DataFrame(columns = col_names)

In [49]:
#loop through all analytes in set
for a in range(0, len(analytes)):
    print(analytes[a])

    #create dataframe for indiv. analyte and its IS
    analyte_df = data.loc[(data['Molecule Name']  == analytes[a]) & (data['Isotope Label Type']  == 'light')]
    analyte_df.reset_index(inplace=True, drop=True)
    
    if ms_type == "Sciex":
        is_df = data.loc[(data['Molecule Name']  == internal_stds[a]) & (data['Isotope Label Type']  == 'heavy')]
        is_df.reset_index(inplace=True, drop=True)
    
    if ms_type == "Waters":
        is_df = data.loc[(data['Transition Note']  == internal_stds[a]) & (data['Isotope Label Type']  == 'heavy')]
        is_df.reset_index(inplace=True, drop=True)
        if (is_df.shape[0] == 0 and data_type == "LC2"):
            is_df = data.loc[(data['Molecule Name']  == internal_stds[a])]
            is_df.reset_index(inplace=True, drop=True)

    #check that analyte and IS dataframes samples are in order
    if(analyte_df['Replicate'].equals(is_df['Replicate'])):
        print('Analyte and Internal Standard dataframes match')
    else:
        print('you got a problem, Analyte and Internal Standard dataframes do NOT match')

    #creat blank temp dataframe
    temp_df = pd.DataFrame(columns = col_names)

    #transfer columns
    temp_df['Analyte Peak Name'] = analyte_df['Molecule Name']
    temp_df['Sample ID'] = analyte_df['Replicate']
    temp_df['Sample ID'] = temp_df['Sample ID'].str.split('_', -1).str[5]
    temp_df['Analyte Retention Time (min)'] = analyte_df['Molecule Retention Time']
    temp_df['Sample Type'] = analyte_df['Sample Type']
    temp_df['Analyte Peak Area (counts)'] = analyte_df['Total Area']
    temp_df['Area Ratio'] = analyte_df['Total Area'] / is_df['Total Area']
    temp_df['Dilution Factor'] = analyte_df['Sample Dilution Factor']
    temp_df['Analyte Concentration (uM)'] = analyte_df['Analyte Concentration']
    temp_df['Calculated Concentration (uM)'] = analyte_df['Calculated Concentration']
    temp_df['Accuracy (%)'] = analyte_df['Accuracy']
    try:
        temp_df['Accuracy (%)'] = temp_df['Accuracy (%)'].str.replace("%","")
    except:
         print('No accuracy found')
    temp_df['File Name'] = analyte_df['File Name']
    temp_df['Acquisition Date'] = analyte_df['Acquired Time']
    temp_df['Acquisition Date'] = temp_df['Acquisition Date'].str.split(' ', -1).str[1]
    temp_df['Analyte Peak Height (cps)'] = analyte_df['Max Height']
    temp_df['Analyte Peak Width (min)'] = analyte_df['Fwhm']
    temp_df['IS Peak Name'] = is_df['Molecule Name']
    temp_df['IS Retention Time (min)'] = is_df['Molecule Retention Time']
    temp_df['IS Peak Area (counts)'] = is_df['Total Area']
    temp_df['IS Concentration (uM)'] = 1
    temp_df['IS Peak Height (cps)'] = is_df['Max Height']
    temp_df['IS Peak Width (min)'] = is_df['Fwhm']
    temp_df['Height Ratio'] = analyte_df['Max Height'] / is_df['Max Height']
    temp_df['Acquisition Method'] = acq_method
    temp_df['Analyte Units'] = "uM"
    temp_df['IS Units'] = "uM"
    temp_df['Plate Position'] = analyte_df['Replicate']
    temp_df['Plate Position'] = temp_df['Plate Position'].str.split('_', -1).str[0]
    temp_df['Rack Position'] = analyte_df['Replicate']
    temp_df['Rack Position'] = temp_df['Rack Position'].str.split('_', -1).str[2]
    temp_df['Analyte Mass Ranges (Da)'] = ""
    temp_df['Analyte Signal To Noise'] = ""
    temp_df['IS Signal To Noise'] = ""
    temp_df['Relative Retention Time'] = analyte_df['Molecule Retention Time']
    temp_df['Sample Name'] = analyte_df['Replicate']
    temp_df['Sample Name'] = temp_df['Sample Name'].str.split('_', -1).str[6]
    #assign Sample Type from ID
    final_data.loc[final_data['Sample ID'] == blank, 'Sample Type'] = "Blank"
    final_data.loc[final_data['Sample ID'] == double_blank, 'Sample Type'] = "Double Blank" 
    
    final_data = pd.concat([final_data, temp_df])
    print('-------------------------')

1-Met-His
Analyte and Internal Standard dataframes match
-------------------------
3-IAA
Analyte and Internal Standard dataframes match
-------------------------
3-IPA
Analyte and Internal Standard dataframes match
-------------------------
3-Met-His
Analyte and Internal Standard dataframes match
-------------------------
5-AVA
Analyte and Internal Standard dataframes match
-------------------------
AABA
Analyte and Internal Standard dataframes match
-------------------------
Ac-Orn
Analyte and Internal Standard dataframes match
-------------------------
ADMA
Analyte and Internal Standard dataframes match
-------------------------
Ala
Analyte and Internal Standard dataframes match
-------------------------
alpha-AAA
Analyte and Internal Standard dataframes match
-------------------------
Anserine
Analyte and Internal Standard dataframes match
-------------------------
Arg
Analyte and Internal Standard dataframes match
-------------------------
Asn_2
Analyte and Internal Standard datafr

In [56]:
#fix formatting
final_data.replace([np.inf, -np.inf], np.nan, inplace=True)
final_data['Calculated Concentration (uM)'] = pd.to_numeric(final_data['Calculated Concentration (uM)'], errors='coerce')
final_data['Height Ratio'] = pd.to_numeric(final_data['Height Ratio'], errors='coerce')
final_data['Area Ratio'] = pd.to_numeric(final_data['Area Ratio'], errors='coerce')

In [57]:
final_data.shape

(8094, 31)

In [58]:
final_data.to_csv(fc2.selected_path + '/' + data_type + "_final_data.csv", index=False)

In [59]:
final_data.to_csv(fc2.selected_path + '/' + data_type + "_final_data.txt", sep='\t', index=False)